# **FRAMEWORKS DE DEEPLEARNING**
# **Prática - Keras**
**Autor**: Renan Santos Mendes

**Email**: renansantosmendes@gmail.com

**Descrição**: Este notebook apresenta um exemplo de uma rede neural profunda com mais de uma camada para um problema de classificação.


# **Saúde Fetal**

As Cardiotocografias (CTGs) são opções simples e de baixo custo para avaliar a saúde fetal, permitindo que os profissionais de saúde atuem na prevenção da mortalidade infantil e materna. O próprio equipamento funciona enviando pulsos de ultrassom e lendo sua resposta, lançando luz sobre a frequência cardíaca fetal (FCF), movimentos fetais, contrações uterinas e muito mais.

Este conjunto de dados contém 2126 registros de características extraídas de exames de Cardiotocografias, que foram então classificados por três obstetras especialistas em 3 classes:

- Normal
- Suspeito
- Patológico

# 1 - Importando os módulos necessários

In [ ]:
!pip install tensorflow

  Using cached protobuf-6.33.1-cp39-abi3-macosx_10_9_universal2.whl.metadata (593 bytes)
Using cached protobuf-6.33.1-cp39-abi3-macosx_10_9_universal2.whl (427 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.27.2
    Uninstalling protobuf-5.27.2:
      Successfully uninstalled protobuf-5.27.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.64.1 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.33.1 which is incompatible.
grpcio-health-checking 1.64.1 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 6.33.1 which is incompatible.


In [ ]:
import os
import tensorflow
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, InputLayer, Dropout, Input
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.regularizers import l1, l2

import os
import random
import numpy as np
import random as python_random
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Definindo funções adicionais

In [ ]:
def plot_convergence(hist, metric):

  df = pd.DataFrame(hist.history)
  fig = px.line(df,
                x=np.arange(df[metric].shape[0]),
                y=[f'{metric}', f'val_{metric}'])
  fig.show()

def reset_seeds():
   os.environ['PYTHONHASHSEED']=str(42)
   tf.random.set_seed(42)
   np.random.seed(42)
   random.seed(42)

# 2 - Fazendo a leitura do dataset e atribuindo às respectivas variáveis

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/renansantosmendes/lectures-cdas-2023/master/fetal_health.csv')

In [ ]:
data.head()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.0,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.0,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.0,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0


# 3 - Preparando o dado antes de iniciar o treino do modelo

In [ ]:
X=data.drop(["fetal_health"], axis=1)
y=data["fetal_health"] - 1
# subtracting -1 from all fetal_health data points == 1 -> 0 // 2 -> 1 // 3 -> 2
# this is because keras.toCategorical starts to count by 0


columns_names = list(X.columns)
X_df = preprocessing.StandardScaler().fit_transform(X)
X_df = pd.DataFrame(X_df, columns=columns_names)

X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.3, random_state=42)


y_train=keras.utils.to_categorical(y_train)
y_test=keras.utils.to_categorical(y_test)

# 4 - Criando o modelo e adicionando as camadas

In [ ]:
# check number of columns to put in layers. Input shape
# X_train.head()
# X_train.shape

data_shape = X_train.iloc[0].shape
data_shape

(21,)

In [ ]:
reset_seeds()

model = keras.Sequential(
    [
        keras.layers.Input(shape=data_shape),
        keras.layers.Dense(32, activation="relu", name="layer1"),
        keras.layers.Dense(32, activation="relu", name="layer2"),
        keras.layers.Dense(32, activation="relu", name="layer3"),
        keras.layers.Dense(3, activation = "softmax", name="outputLayer"),
    ]
)

#this is another way to declare the model strucutre
# model = keras.Sequential()
# model.add(InputLayer())
# ...

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ layer1 (Dense)                  │ (None, 32)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer2 (Dense)                  │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer3 (Dense)                  │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ outputLayer (Dense)             │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,915 (11.39 KB)

 Trainable params: 2,915 (11.39 KB)

 Non-trainable params: 0 (0.00 B)

# 5 - Compilando o modelo


In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"] )

# 6 - Executando o treino do modelo

In [ ]:
#validation_split is the "fold" to cross-validation
hist = model.fit(x=X_train,
    y=y_train,
    batch_size=120,
    epochs=10,
    validation_split=0.2,
    shuffle=True,
)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9963 - loss: 0.0289 - val_accuracy: 0.9295 - val_loss: 0.3316
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9963 - loss: 0.0284 - val_accuracy: 0.9329 - val_loss: 0.3310
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9963 - loss: 0.0276 - val_accuracy: 0.9329 - val_loss: 0.3363
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9951 - loss: 0.0272 - val_accuracy: 0.9295 - val_loss: 0.3349
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9963 - loss: 0.0263 - val_accuracy: 0.9295 - val_loss: 0.3377
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9963 - loss: 0.0260 - val_accuracy: 0.9329 - val_loss: 0.3399
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9963 - loss: 0.0252 - val_accuracy: 0.9295 - val_loss: 0.3405
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9963 - loss: 0.0252 - val_accuracy: 0.9295 - v

# 6.1 Avaliando o modelo

In [ ]:
model.evaluate(
    x=X_test,
    y=y_test,
    batch_size=None,
)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9155 - loss: 0.3729 


[0.36049535870552063, 0.9153605103492737]

# 6.2 Plotando a acurácia do modelo

In [ ]:
plot_convergence(hist, "loss")